In [ ]:
"""
Purpose: To test out the changes needed to make to datajoint table 
for v4 run: 

1) changing how the axon and dendrite skeleton are saved
2) Doing the complete axon preprocessing, saving that off
and then passing that to the neuron proofreading function
3) How to save off the suggested edits in form that we 
can create suggested edits fo rneuroglancer
4) Make sure webbing and bouton information in table
"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty")

import datajoint_utils as du
from importlib import reload

INFO - 2021-05-17 21:55:25,712 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-17 21:55:25,714 - settings - Setting database.user to celiib
INFO - 2021-05-17 21:55:25,715 - settings - Setting database.password to newceliipass
INFO - 2021-05-17 21:55:25,718 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-05-17 21:55:25,718 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-17 21:55:25,729 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-05-17 21:55:26,129 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-17 21:55:26,131 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-17 21:55:26,132 - settings - Setting database.user to celiib
INFO - 2021-05-17 21:55:26,133 - settings - Setting database.password to newceliipass
INFO - 2021-05-17 21:55:26,134 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-17 21:55:26,134 - settings - Setting database.user to celiib
INFO - 2021-05-17 21:55:26,135 - settings - Setting database.password to newceliipass
INFO - 2021-05-17 21:55:26,135 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-17 21:55:26,557 - settings - Setting enable_python_native_blobs to True


In [3]:
#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2021-05-17 21:55:26,683 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-17 21:55:26,684 - settings - Setting database.user to celiib
INFO - 2021-05-17 21:55:26,685 - settings - Setting database.password to newceliipass
INFO - 2021-05-17 21:55:26,686 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-17 21:55:27,276 - settings - Setting enable_python_native_blobs to True


In [4]:
import neuron_visualizations as nviz
import neuron_utils as nru
import proofreading_utils as pru
import time
import system_utils as su
import axon_utils as au
import trimesh_utils as tu

# Experimenting With New Way of Saying Skeletons and faces

In [ ]:
"""
5/17 
Purpose: To now save off the skeletons and mesh faces as numpy objects

"""

In [12]:
x = np.random.rand(10)
np.savez_compressed("test_np_compressed.npz",x = x)
x

array([0.61460661, 0.07557948, 0.459777  , 0.16375646, 0.19806134,
       0.64466776, 0.43616094, 0.02041192, 0.65352067, 0.61322128])

In [19]:
filename = Path("test_np_compressed.npz")
x_reload = np.load(filename)["x"]
x_reload

INFO - 2021-05-17 22:05:39,508 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-17 22:05:39,510 - settings - Setting database.user to celiib
INFO - 2021-05-17 22:05:39,511 - settings - Setting database.password to newceliipass
INFO - 2021-05-17 22:05:39,512 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-17 22:05:39,512 - settings - Setting database.user to celiib
INFO - 2021-05-17 22:05:39,513 - settings - Setting database.password to newceliipass
INFO - 2021-05-17 22:05:39,518 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-17 22:05:40,230 - settings - Setting enable_python_native_blobs to True


array([0.61460661, 0.07557948, 0.459777  , 0.16375646, 0.19806134,
       0.64466776, 0.43616094, 0.02041192, 0.65352067, 0.61322128])

In [21]:
minnie,schema = du.configure_minnie_vm()
mesh_faces = (minnie.AutoProofreadStats4 & dict(segment_id = 864691134884759034)).fetch("mesh_faces")

INFO - 2021-05-17 22:18:06,536 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-17 22:18:06,539 - settings - Setting database.user to celiib
INFO - 2021-05-17 22:18:06,541 - settings - Setting database.password to newceliipass
INFO - 2021-05-17 22:18:06,542 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-17 22:18:06,544 - settings - Setting database.user to celiib
INFO - 2021-05-17 22:18:06,546 - settings - Setting database.password to newceliipass
INFO - 2021-05-17 22:18:06,586 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-17 22:18:09,734 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-17 22:18:11,170 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-17 22:18:11,179 - settings - Setting database.user to celiib
INFO - 2021-05-17 22:18:11,179 - settings - Setting database.password to newceliipass
INFO - 2021-05-17 22:18:11,185 - settings - Setting enable_pytho

filepath = /mnt/dj-stor01/platinum/minnie65/02/glia_nuclei_faces/864691134884759034_0_proofv4_neuron_proofread.pbz2


In [20]:
from pathlib import Path
filepath = Path("/mnt/dj-stor01/platinum/minnie65/02/glia_nuclei_faces/864691134884759034_0_proofv4_neuron_proofread.pbz2")
filepath.suffix  == ".pbz2"

INFO - 2021-05-17 22:06:35,880 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-17 22:06:35,881 - settings - Setting database.user to celiib
INFO - 2021-05-17 22:06:35,881 - settings - Setting database.password to newceliipass
INFO - 2021-05-17 22:06:35,882 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-17 22:06:35,882 - settings - Setting database.user to celiib
INFO - 2021-05-17 22:06:35,883 - settings - Setting database.password to newceliipass
INFO - 2021-05-17 22:06:35,885 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-17 22:06:36,509 - settings - Setting enable_python_native_blobs to True


True

# Debugging the Splitting Errors

In [ ]:
[k for k in dir(tu) if "box" in k]

In [ ]:
neuron_split_idxs

In [ ]:
segment_id = 864691136540183458
segment_id = 864691136136605053
segment_id = 864691136464362887
segment_id = 864691135791213391
segment_id = 864691136031747259
segment_id = 864691135841099491
segment_id = 864691135538151026
neuron_objs,neuron_split_idxs = du.decomposition_with_spine_recalculation(segment_id)
neuron_obj = neuron_objs[0]

In [ ]:
curr_limb_mesh= su.decompress_pickle("curr_limb_mesh")
miss_mesh= su.decompress_pickle("miss_mesh")
dec_mesh= su.decompress_pickle("dec_mesh")

In [ ]:
nviz.plot_objects(dec_mesh)

In [ ]:
split_results = pru.multi_soma_split_suggestions(orig_neuron_obj,plot_intermediates=False)

In [ ]:
verbose = True
(neuron_list,
        neuron_list_errored_limbs_area,
         neuron_list_errored_limbs_skeletal_length,
        neuron_list_n_multi_soma_errors,
        neuron_list_n_same_soma_errors) = pru.split_neuron(orig_neuron_obj,
                        limb_results=split_results,
                                       verbose=verbose,
                                        return_error_info=True
                                            )

In [ ]:
ret_path = neuron_list[1].save_compressed_neuron("./",
                                      export_mesh=True,
                                     return_file_path=True,
                                     )

In [ ]:
nviz.plot_objects()

In [ ]:
nviz.plot_objects(miss_mesh,
                 main_mesh_color="red",
                  show_at_end=False,
                 append_figure=True,
                  
                )

In [ ]:
orig_neuron_obj_path = (minnie.Decomposition & dict(segment_id=segment_id)).fetch1("decomposition")
orig_neuron_obj = du.filepath_to_neuron_obj(orig_neuron_obj_path)

In [ ]:
nviz.visualize_neuron_lite(orig_neuron_obj)

In [ ]:
orig_neuron_obj.n_somas

In [ ]:
axon_mesh_filtered_idx = tu.filter_meshes_by_containing_coordinates(diff_meshes,
                                              nullifying_points=limb_obj.current_touching_soma_vertices,
                                               method="distance",
                                               distance_threshold = 0,
                                              filter_away=False,
                                            return_indices=True,
                                                                   verbose=True)
axon_mesh_filtered_idx

In [ ]:
diff_meshes

In [ ]:
import preprocessing_vp2 as pre
pre.high_fidelity_axon_decomposition(neuron_obj)

# BBox function

In [ ]:
"""
Purpose: To test whether a certain number of points line within a bounding box 

"""

# Need a table to store the fully axon processed neurons

In [ ]:
minnie.AutoProofreadVersion.describe()

# Testing Populating the Axon Decomposition Table

In [ ]:
verbose = True

In [ ]:
curr_table = (minnie.Decomposition() & "n_limbs = 10" & "n_somas=2")
curr_table

In [ ]:
key = (curr_table & dict(segment_id = 864691135012573174)).proj().fetch1()
key

In [ ]:
verbose = True
# 1) Pull Down All of the Neurons
segment_id = key["segment_id"]

whole_pass_time = time.time()

#1) Pull Down all the Neuron Objects associated with a segment_id

neuron_objs,neuron_split_idxs = du.decomposition_with_spine_recalculation(segment_id)

if verbose:
    print(f"Number of Neurons found ={len(neuron_objs)}")
    
#For each neuron:
#for split_index,neuron_obj_pre_split in zip(neuron_split_idxs,neuron_objs):


In [ ]:
split_index = 1
neuron_obj = neuron_objs[split_index]

#Run the Axon Decomposition
neuron_obj_with_web = au.complete_axon_processing(neuron_obj,
                                     verbose=True)

In [ ]:
neuron_obj_with_web = n_obj

In [ ]:
# Just trying to filter neuron to se what it would look like
filter_list = pru.v4_exc_filters()
inh_exc_class="excitatory"
perform_axon_classification = False
plot_limb_branch_filter_with_disconnect_effect = True

filtered_neuron,filtering_info = pru.proofread_neuron_class_predetermined(neuron_obj_with_web,
        inh_exc_class=inh_exc_class,
        perform_axon_classification=False,

        plot_limb_branch_filter_with_disconnect_effect = plot_limb_branch_filter_with_disconnect_effect,


        plot_final_filtered_neuron = False,

        #arguments for the axon high fidelity:
        plot_new_axon_limb_correspondence = False,
        plot_new_limb_object = False,
        plot_final_revised_axon_branch = False,

        verbose = False,
        verbose_outline = verbose,
        filter_list = filter_list,
        high_fidelity_axon_on_excitatory = perform_axon_classification,
                                                                             )

In [ ]:
nviz.plot_original_vs_proofread(original = neuron_obj,
    proofread = filtered_neuron,
    original_color = "red",
    proofread_color = "blue",
    mesh_alpha = 0.3,
    plot_mesh= True,
    plot_skeleton = True)

In [ ]:
filtered_neuron.description

In [ ]:
minnie.Decomposition() & "n_somas = 2"

In [ ]:
minnie.DecompositionAxon()

In [ ]:
minnie.DecompositionAxon() & "axon_length>1000" & "segment_id=864691134917425418"

In [ ]:
n_obj = du.decomposition_with_spine_recalculation(864691134917425418,0)

In [ ]:
orig_seg_mesh = du.fetch_segment_id_mesh(n_obj.segment_id)

In [ ]:
orig_seg_mesh

In [ ]:
nviz.plot_original_vs_proofread(original = n_obj,
    proofread = filtered_neuron,
    original_color = "red",
    proofread_color = "blue",
    mesh_alpha = 0.3,
    plot_mesh= True,
    plot_skeleton = False)

In [ ]:
x = None
try:
    du.decomposition_with_spine_recalculation(864691136231316382,0)
except Exception as e:
    x = e.args[0]
    print(e)

In [ ]:
import datajoint_utils as du
du.datajoint_filepath_error_to_filepath(error = x)

In [ ]:
save_time = time.time()
ret_file_path = filtered_neuron.save_compressed_neuron(
                                #output_folder=str(du.get_decomposition_path()),
                                output_folder = "./",
                                #file_name=f"{neuron_obj_with_web.segment_id}_validation_full_axon",
                                  return_file_path=True,
                                 export_mesh=False,
                                 suppress_output=False)

ret_file_path_str = str(ret_file_path.absolute()) + ".pbz2"
print(f"ret_file_path_str = {ret_file_path_str}")
print(f"Save time = {time.time() - save_time}")

In [ ]:
n_dict = dict(key,
              split_index = split_index,
              axon_version = au.axon_version,
             decomposition=ret_file_path_str,
             axon_length=filtered_neuron.axon_length,
              run_time = np.round(time.time() - whole_pass_time,2)
             )

n_dict

# Fetching an example neuron

In [ ]:
du.decomposition_with_spine_recalculation?

In [ ]:
table_to_neuron_objs(table,
    return_one=False,
    return_split_index=False)

In [ ]:
du.table_to_neuron_objs

In [ ]:
curr_table = minnie.DecompositionSplit() & "segment_id = 864691134917398026"
curr_table

In [ ]:
"""
Need a function that will go and get the 


"""
